In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')


In [2]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [3]:
from typing_extensions import List, TypedDict


class State(TypedDict):
    story_so_far: str
    user_input: str
    answer: str



In [4]:
from langchain.prompts import ChatPromptTemplate

story_prompt = ChatPromptTemplate.from_messages([
    ("system",
     """You are a collaborative storyteller AI. 
     You and the user take turns writing a single paragraph of an ongoing story.
     
     Rules:
     - Always continue from the last events in the story so far.
     - Keep tone, style, and pacing consistent with earlier parts.
     - Write only one paragraph per turn.
     - End your paragraph in a way that invites the user to continue (e.g., suspense, open-ended action).
     - Do NOT rewrite or summarize the user's part — just continue from it.
     """),
    
    ("human", 
     """Story so far:
{story_so_far}

User's turn:
{user_input}

Now it's your turn to write the next paragraph of the story.""")
])


In [10]:
from langchain.prompts import PromptTemplate

story_so_far_prompt = PromptTemplate(
    input_variables=["user_input"],
    template=(
        "You are a skilled storyteller across all genres.\n"
        "Based solely on the user's input, infer:\n"
        "1. The likely **genre** (fantasy, mystery, romance, sci-fi, horror, historical, comedy, etc.).\n"
        "2. The **setting** (place, time, atmosphere) suggested by the input.\n"
        "3. The **tone** (serious, whimsical, dark, hopeful, etc.).\n\n"
        "Then write a vivid and immersive 'Story So Far' as the opening scene.\n"
        "Guidelines:\n"
        "- Keep it under 200 words.\n"
        "- Do not explicitly label the genre, setting, or tone — just show them through the prose.\n"
        "- Make the scene feel alive with sensory details and a hook.\n\n"
        "User's input: {user_input}\n\n"
        "Story so far:"
    )
)


In [11]:
def set_story(state: State):
  story = story_so_far_prompt.invoke({"user_input":state['user_input']})
  respone = llm.invoke(story)
  return {"story_so_far":respone.content}

In [ ]:
def generate(state: State):
    story = story_prompt.invoke({"story_so_far":state['story_so_far'],"user_input":state['user_input']})
    response = llm.invoke(story)
    return {"answer": response.content,"story_so_far":state['story_so_far']+"\n"+state['user_input']+"\n"+response.content}

In [ ]:
from langgraph.graph import START, StateGraph

graph_builder = StateGraph(State)
graph_builder.add_sequence([set_story, generate])
graph_builder.add_edge(START, "set_story")
graph = graph_builder.compile()


In [12]:
from dotenv import load_dotenv
import os

load_dotenv()

# Initial state
state = {
    "user_input": "",
    "story_so_far": "",
    "answer": ""
}

print("\n--- AI Story Generator ---")
print("Type 'quit' to end the game.\n")

# First turn (opening idea)
user_input = input("Enter your opening idea: ")
state["user_input"] = user_input
state = graph.invoke(state)

# Print AI's first story so far
print(f"\nAI's opening:\n{state['story_so_far']}")

# Game loop
while True:
    user_input = input("\nYour turn: ")

    if user_input.lower().strip() == "quit":
        print("\nThanks for playing!")
        break

    # Update state with user input
    state["user_input"] = user_input

    # Run through the graph
    state = graph.invoke(state)

    # Show AI's turn
    print(f"\nAI's turn:\n{state['answer']}")



--- AI Story Generator ---
Type 'quit' to end the game.


AI's opening:
The air itself was a gritty reminder of the bombs, perpetually thick with dust and the faint tang of irradiated decay, making every breath a conscious effort. Scavenging for even a single can of nutrient paste meant risking encounters with the 'Shriekers' – mutated horrors that hunted by sound, their guttural cries echoing across the shattered ruins of what was once a sprawling city. Yet, the monsters were often less terrifying than the desperate eyes of another survivor, glimpsed from afar, knowing that any encounter would likely end in a brutal fight over meager scraps, or worse, a forced recruitment into a tyrannical settlement. It was a world where trust was a forgotten luxury, and every sunrise brought with it the same gnawing question: how much longer could one truly survive alone?

AI's turn:
Furiosa had heard the whispers of the fabled city of Elysium since she was a child, a sanctuary supposedly untouched